## Decode Linear Page Address to Page, Block, Plane Index

In [1]:
def decode_page_address(page_address: int, pages_per_block: int, planes: int) -> tuple[int, int, int]:
    """
    Decode a linear page address into (page, block, plane_index)
    
    :param page_address:     The straight page address.
    :param pages_per_block:  Number of pages in each block.
    :param planes:           Number of planes (1, 2, 4, ...).
    
    :return: (page, block, plane_index)
    """
    # 1) Extract 'page' within a block:
    page = page_address % pages_per_block
    
    # 2) Remove 'page' bits to get the combined block/plane field:
    block_plane_field = page_address // pages_per_block
    
    # 3) Extract 'plane_index' from the lower bits:
    #    plane_index = (block_plane_field mod planes) 
    #    because if planes=2 => 1 bit; if planes=4 => 2 bits, etc.
    plane_index = block_plane_field % planes
    
    # 4) "pure block" index :
    block = block_plane_field
    
    #    If you prefer to keep plane bits inside 'block', then do:
    # block = block_plane_field
    
    return (page, block, plane_index)

## Build Address (5 or 3 Cycles)

In [2]:
def build_5_cycle_address(column: int, page: int, block: int, plane: int, lun: int) -> list[int]:
    """
    Build 5 address cycles (bytes) according to the layout:
    1) Column address (16 bits):
       - Cycle 1 => column[7..0]
       - Cycle 2 => column[15..8]
    2) Page address (10 bits total):
       - Cycle 3 => page[7..0]        (lowest 8 bits)
       - Cycle 4 => page[9..8]       (bits [1..0])
    3) Plane (2 bits), embedded in BA[11..10]:
       - Cycle 4 bits [3..2] => plane[1..0]
    4) Block address (10 bits), combined with plane bits => total 12 bits:
       - Cycle 4 bits [7..4] => block[3..0]   (changed as requested)
       - Cycle 5 bits [5..0] => block[9..4]   (changed as requested)
    5) LUN (1 bit):
       - Cycle 5 bit [6] => lun
    :param column: up to 16 bits (0..65535)
    :param page:   up to 10 bits (0..1023)
    :param block:  up to 10 bits (0..1023) – plane bits separate
    :param plane:  0..3 (2 bits)
    :param lun:    0..1 (1 bit)
    :return: List of 5 bytes [cycle1..cycle5].
    """

    # ---------------------
    # Cycle 1 => lower 8 bits of column
    # Cycle 2 => upper 8 bits of column
    # ---------------------
    c1 = column & 0xFF
    c2 = (column >> 8) & 0xFF
    
    # ---------------------
    # Cycle 3 => lower 8 bits of page: page[7..0]
    # ---------------------
    c3 = page & 0xFF
    
    # ---------------------
    # Cycle 4 => c4[7..0]:
    #   bits[7..4] -> block[3..0] 
    #   bits[3..2] -> plane[1..0]
    #   bits[1..0] -> page[9..8]
    # ---------------------
    c4 = 0
    # block[3..0] => lower 4 bits of block
    c4 |= (block & 0xF) << 4
    # plane => 2 bits in [3..2]
    c4 |= (plane & 0x3) << 2
    # top 2 bits of page => page[9..8]
    c4 |= (page >> 8) & 0x3
    
    
    # ---------------------
    # Cycle 5 => c5[7..0]:
    #   bits[5..0] -> block[9..4]
    #   bit [6]    -> lun
    #   bit [7]    -> 0
    # ---------------------
    c5 = 0
    # block[9..4] => 6 bits
    c5 |= ((block >> 4) & 0x3F)
    # lun => bit [6]
    c5 |= ((lun & 0x1) << 6)
    
    return [c1, c2, c3, c4, c5]

In [3]:
def build_3_cycle_address(page: int, block: int, plane: int, lun: int) -> list[int]:
    
    # ---------------------
    # Cycle 1 => lower 8 bits of page: page[7..0]
    # ---------------------
    c1 = page & 0xFF
    
    # ---------------------
    # Cycle 2 => c4[7..0]:
    #   bits[7..4] -> block[3..0] 
    #   bits[3..2] -> plane[1..0]
    #   bits[1..0] -> page[9..8]
    # ---------------------
    c2 = 0
    # block[3..0] => lower 4 bits of block
    c2 |= (block & 0xF) << 4
    # plane => 2 bits in [3..2]
    c2 |= (plane & 0x3) << 2
    # top 2 bits of page => page[9..8]
    c2 |= (page >> 8) & 0x3
    
    
    # ---------------------
    # Cycle 3 => c5[7..0]:
    #   bits[5..0] -> block[9..4]
    #   bit [6]    -> lun
    #   bit [7]    -> 0
    # ---------------------
    c3 = 0
    # block[9..4] => 6 bits
    c3 |= ((block >> 4) & 0x3F)
    # lun => bit [6]
    c3 |= ((lun & 0x1) << 6)
    
    return [c1, c2, c3]

## Commands for Nand Flash and FTDI

In [4]:
#FTDI Commands
ACTIVATE_COMMNAD_LATCH = 0x40
ACTIVATE_ADDRESS_LATCH = 0x80
ACTIVATE_WRITE_PROTECT = 0x20
DEACTIVATE_CA_LATCH = 0x00

#Nand Flash Commands
NAND_CMD_READ_CYCLE1 = 0x00
NAND_CMD_READ_CYCLE2 = 0x30

NAND_CMD_PROGRAM_CYCLE1 = 0x80
NAND_CMD_PROGRAM_CYCLE2 = 0x10

NAND_CMD_BLOCK_ERASE_CYCLE1 = 0x60
NAND_CMD_BLOCK_ERASE_CYCLE2 = 0xD0

NAND_CMD_RESET = 0xFF
NAND_CMD_READ_STATUS = 0x70

WR_ADDRESS_CYCLE = 5
ERASE_ADDRESS_CYCLE = 3

## Nand Read Page Functions

In [5]:
def read_nand_command (cycle):
    #Command Cycle 1
    cmds1 = [Ftdi.WRITE_EXTENDED, ACTIVATE_COMMNAD_LATCH, 0] 
    cmds1 += [NAND_CMD_READ_CYCLE1]
    ftdi.write_data(Array('B', cmds1))
    
    #Address Cycle 5 Consequtive
    cmds2 = [Ftdi.WRITE_EXTENDED, ACTIVATE_ADDRESS_LATCH, 0 , cycle[0]]
    for i in range(1,WR_ADDRESS_CYCLE,1):
        cmds2 += [Ftdi.WRITE_SHORT, 0, cycle[i]]
    ftdi.write_data(Array('B', cmds2))
    
    #Command Cycle 2
    cmds3 = [Ftdi.WRITE_EXTENDED, ACTIVATE_COMMNAD_LATCH, 0]
    cmds3 += [NAND_CMD_READ_CYCLE2]
    ftdi.write_data(Array('B', cmds3))
    
    
    print(f"Command 1:  {cmds1}")
    print(f"Command 2:  {cmds2}")
    print(f"Command 3:  {cmds3}")

In [6]:
def read_nand_page_bytes(read_len):
    cmds = []
    
    #Data Read Cycle
    cmds += [ftdi.READ_EXTENDED, DEACTIVATE_CA_LATCH , 0]
    
    for i in range(1, read_len, 1):
        cmds += [Ftdi.READ_SHORT, 0]
        
    cmds.append(Ftdi.SEND_IMMEDIATE)
    ftdi.write_data(Array('B', cmds))
    data = ftdi.read_data_bytes(read_len)
    return bytes(data)

## NAND Write Page Functions

In [7]:
def program_nand_page(cycle, data):
    #Command Cycle 1
    activateCMD = ACTIVATE_WRITE_PROTECT | ACTIVATE_COMMNAD_LATCH
    cmds1 = [Ftdi.WRITE_EXTENDED, activateCMD, 0] 
    cmds1 += [NAND_CMD_PROGRAM_CYCLE1]
    
    #Address Cycle 5 Consequtive
    activateADR = ACTIVATE_WRITE_PROTECT | ACTIVATE_ADDRESS_LATCH
    cmds2 = [Ftdi.WRITE_EXTENDED, activateADR, 0 , cycle[0]]
    for i in range(1,WR_ADDRESS_CYCLE,1):
        cmds2 += [Ftdi.WRITE_SHORT, 0, cycle[i]]
    
    #data cycle 
    activateDTC = ACTIVATE_WRITE_PROTECT | DEACTIVATE_CA_LATCH
    cmds3 = [Ftdi.WRITE_EXTENDED, activateDTC , 0, data[0]]
    for i in range(1,len(data),1):
        cmds3 += [Ftdi.WRITE_SHORT, 0, data[i]]
    
    #Command Cycle 2
    activateCMD = ACTIVATE_WRITE_PROTECT | ACTIVATE_COMMNAD_LATCH
    cmds4 = [Ftdi.WRITE_EXTENDED, activateCMD, 0]
    cmds4 += [NAND_CMD_PROGRAM_CYCLE2]
    
    #print(f"Command 1:  {cmds1}")
    #print(f"Command 2:  {cmds2}")
    #print(f"Command 3:  {cmds3}")
    #print(f"Command 4:  {cmds4}")
    
    ftdi.write_data(Array('B', cmds1))
    time.sleep(0.01)
    ftdi.write_data(Array('B', cmds2))
    time.sleep(0.01)
    ftdi.write_data(Array('B', cmds3))
    time.sleep(0.01)
    ftdi.write_data(Array('B', cmds4))
    time.sleep(0.01)

In [8]:
def status_register_check():
    #Staus Cycle
    cmds1 = [Ftdi.WRITE_EXTENDED, ACTIVATE_COMMNAD_LATCH, 0]
    cmds1 += [NAND_CMD_READ_STATUS]
    ftdi.write_data(Array('B', cmds1))
    #print(f"Command 5:  {cmds1}")
        
    cmds1 = [ftdi.READ_EXTENDED, DEACTIVATE_CA_LATCH , 0]
    cmds1 += [Ftdi.READ_SHORT, 0]
        
    cmds1.append(Ftdi.SEND_IMMEDIATE)
    ftdi.write_data(Array('B', cmds1))
    
    status = ftdi.read_data_bytes(1)
    state = ord(status)
    
    print(f"Status:{state:08b}")
    if state == 96:
        print("Successfully Done!!")
    else:
        print("failed, check the status byte")

In [9]:
def createDataStream(sizeDataArray):
    dataStream = []
    data = 0
    for i in range(0, sizeDataArray, 1):
        data += 1
        dataStream.append(data)
        if (data >= 8):
            data = 0
    print(len(dataStream))
    return dataStream

In [32]:
def createDataStreamZeros(sizeZerosArray):
    dataStream = []
    for i in range(0, sizeZerosArray, 1):
        dataStream.append(0)
    #print(len(dataStream))
    #print(dataStream)
    return dataStream

In [11]:
def createCharacterStream(sizechrArray):
    chrStream = ""
    data = 0
    for i in range(0, sizechrArray, 1):
        data = "A"
        chrStream += data
    print(len(chrStream))
    print(chrStream)
    return chrStream

## NAND Erase Block Functions

In [12]:
def erase_nand_block (cycle):
    #Command Cycle 1
    activateCMD = ACTIVATE_WRITE_PROTECT | ACTIVATE_COMMNAD_LATCH
    cmds1 = [Ftdi.WRITE_EXTENDED, activateCMD, 0] 
    cmds1 += [NAND_CMD_BLOCK_ERASE_CYCLE1]
    
    #Address Cycle 3 Consequtive
    activateADR = ACTIVATE_WRITE_PROTECT | ACTIVATE_ADDRESS_LATCH
    cmds2 = [Ftdi.WRITE_EXTENDED, activateADR, 0 , cycle[0]]
    for i in range(1,ERASE_ADDRESS_CYCLE,1):
        cmds2 += [Ftdi.WRITE_SHORT, 0, cycle[i]]
    
    #Command Cycle 2
    activateCMD = ACTIVATE_WRITE_PROTECT | ACTIVATE_COMMNAD_LATCH
    cmds3 = [Ftdi.WRITE_EXTENDED, activateCMD, 0]
    cmds3 += [NAND_CMD_BLOCK_ERASE_CYCLE2]
    
    print(f"Command 1:  {cmds1}")
    print(f"Command 2:  {cmds2}")
    print(f"Command 4:  {cmds3}")
    
    ftdi.write_data(Array('B', cmds1))
    ftdi.write_data(Array('B', cmds2))
    ftdi.write_data(Array('B', cmds3))
    time.sleep(1)

## FTDI Initialization

In [13]:
#essential functions
import usb.core
import usb.util
import time
from array import array as Array
from pyftdi.ftdi import *

In [14]:
#essential functions
VENDOR_ID = 0x0403
PRODUCT_ID = 0x6010

# direction_mask: 1 means output, 0 means input
# value_mask:     1 sets the pin high, 0 sets it low
DIRECTION_MASK = 0xFF  # all upper 8 bits as output
VALUE_MASK = 0x00     # pattern to set the pins to

# Initialize FTDI
#Creates an instance or object of the Ftdi class and stored in the ftdi variable
ftdi = Ftdi()

#Open a new interface to the specified FTDI device.
#open() method establishes a connection to a specific FTDI device.
ftdi.open(VENDOR_ID, PRODUCT_ID, interface=1)

#Configures the FTDI device into MCU Host Bus Emulation mode
ftdi.set_bitmode(0x00, Ftdi.BitMode.MCU)  # MCU = 0x08 (MCU Host Bus Emulation mode)

#Disable the “divide-by-5” clock prescaler allows higher frequency operation
ftdi.write_data(Array('B', [Ftdi.DISABLE_CLK_DIV5]))

#the FTDI device waits (in milliseconds) 
#before sending any partially filled USB packet up to the host if the buffer is not yet full.
ftdi.set_latency_timer(2)


# This tells the FTDI chip to set upper 8 bits with these direction and value settings
ftdi.write_data(bytes([
    Ftdi.SET_BITS_HIGH,
    VALUE_MASK,
    DIRECTION_MASK
]))

#Any pending outbound data in the transmit buffer is cleared
ftdi.purge_buffers()

#Reset the the nand flash chip
cmdsRST = [Ftdi.WRITE_EXTENDED, ACTIVATE_COMMNAD_LATCH, 0, NAND_CMD_RESET]
print (cmdsRST)
ftdi.write_data(Array('B', cmdsRST))

[147, 64, 0, 255]


4

# MAIN CODE: Read, Write, Erase Operations of Nand Flash Memory

## Read Page Byte by Byte

In [15]:
#convert linear page address
linearPageAddress = 121
pagesPerBlock = 1024
NumberOfPlane = 4

pageX, blockX, planeX = decode_page_address(linearPageAddress, pagesPerBlock, NumberOfPlane)

print(f"Address {linearPageAddress} => Page {pageX}, Block {blockX}, Plane {planeX}")

Address 121 => Page 121, Block 0, Plane 0


In [16]:
#create address cycle
column = 0
page   = pageX
block  = blockX
plane  = planeX
lun    = 0

#create 5 address cycle
cycles = build_5_cycle_address(column, page, block, plane, lun)
print(f"Address Cycles: {cycles}")

print("5 Address Cycles:")
for i, val in enumerate(cycles, start=1):
    print(f" Cycle {i}: 0x{val:02X}  (bin: {val:08b})")


Address Cycles: [0, 0, 121, 0, 0]
5 Address Cycles:
 Cycle 1: 0x00  (bin: 00000000)
 Cycle 2: 0x00  (bin: 00000000)
 Cycle 3: 0x79  (bin: 01111001)
 Cycle 4: 0x00  (bin: 00000000)
 Cycle 5: 0x00  (bin: 00000000)


In [17]:
byteLength = 100

#Send the command and address cycles before page read
read_nand_command(cycles)

#Read byte by byte from Data Register
read_nand_page_bytes(byteLength)

Command 1:  [147, 64, 0, 0]
Command 2:  [147, 128, 0, 0, 146, 0, 0, 146, 0, 121, 146, 0, 0, 146, 0, 0]
Command 3:  [147, 64, 0, 48]


b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'

## Program Pages in a Block

In [18]:
linearPageAddress = 121
pagesPerBlock = 1024
NumberOfPlane = 4

pageX, blockX, planeX = decode_page_address(linearPageAddress, pagesPerBlock, NumberOfPlane)
addressCycleStream = build_5_cycle_address(0, pageX, blockX, planeX, 0)

In [19]:
pageBytes = 16384
data = createDataStream(pageBytes)

program_nand_page (addressCycleStream, data)

status_register_check()

16384
Status:01100000
Successfully Done!!


In [20]:
ftdi.purge_buffers()
read_nand_command(addressCycleStream)
read_nand_page_bytes(100)

Command 1:  [147, 64, 0, 0]
Command 2:  [147, 128, 0, 0, 146, 0, 0, 146, 0, 121, 146, 0, 0, 146, 0, 0]
Command 3:  [147, 64, 0, 48]


b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x04\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'

## Erase a Block

In [33]:
linearPageAddress = 121
pagesPerBlock = 1024
NumberOfPlane = 4

pageX, blockX, planeX = decode_page_address(linearPageAddress, pagesPerBlock, NumberOfPlane)
print(f"Address {linearPageAddress} => Page {pageX}, Block {blockX}, Plane {planeX}")

blockEraseAddress = build_3_cycle_address(pageX, blockX, planeX, 0)

print("3 Address Cycles:")
for i, val in enumerate(blockEraseAddress, start=1):
    print(f" Cycle {i}: 0x{val:02X}  (bin: {val:08b})")

Address 121 => Page 121, Block 0, Plane 0
3 Address Cycles:
 Cycle 1: 0x79  (bin: 01111001)
 Cycle 2: 0x00  (bin: 00000000)
 Cycle 3: 0x00  (bin: 00000000)


In [34]:
erase_nand_block (blockEraseAddress)

status_register_check()

Command 1:  [147, 96, 0, 96]
Command 2:  [147, 160, 0, 121, 146, 0, 0, 146, 0, 0]
Command 4:  [147, 96, 0, 208]
Status:01100000
Successfully Done!!


In [35]:
ftdi.purge_buffers()
read_nand_command(addressCycleStream)
read_nand_page_bytes(100)

Command 1:  [147, 64, 0, 0]
Command 2:  [147, 128, 0, 0, 146, 0, 0, 146, 0, 255, 146, 0, 3, 146, 0, 0]
Command 3:  [147, 64, 0, 48]


b'\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff'

In [24]:
#ftdi.close();

## Program a block

In [36]:
pagePerBlock = 1024
pageno = 0
blockno = 0
planeno = 0
pageBytes = 16384
for i in range(0, pagePerBlock, 1):
    pageno = i
    addressCycleStream = build_5_cycle_address(0, pageno, blockno, planeno, 0)
    data = createDataStreamZeros(pageBytes)
    program_nand_page (addressCycleStream, data)
status_register_check()

Status:01100000
Successfully Done!!


In [53]:
pagePerBlock = 1024
pageno = 101
blockno = 0
planeno = 0
pageBytes = 16384
max_rx_buffer = 6000
readData = bytes()
ftdi.purge_buffers()

addressCycleStream = build_5_cycle_address(0, pageno, blockno, planeno, 0)
read_nand_command(addressCycleStream)

for i in range(0, pageBytes//max_rx_buffer, 1):
    readData += read_nand_page_bytes(max_rx_buffer)
readData += read_nand_page_bytes(pageBytes%max_rx_buffer)

#Check the Programmed Page
print(len(readData))
print(type(readData))
print(readData[10])

Command 1:  [147, 64, 0, 0]
Command 2:  [147, 128, 0, 0, 146, 0, 0, 146, 0, 101, 146, 0, 0, 146, 0, 0]
Command 3:  [147, 64, 0, 48]
16384
<class 'bytes'>
0
